In [3]:
#%pip install cupy-cuda12x

In [4]:
import cupy as cp
import numpy as np
import random
from typing import List, Tuple, Iterable, Set
from itertools import combinations
from math import inf
import pandas as pd

In [5]:
Point = Tuple[int, int]

def generate_instance(num_rows_columns: int, D: int, J: int, seed: int = 42) -> Tuple[List[Point], List[Point]]:
    if D + J > num_rows_columns * num_rows_columns:
        raise ValueError("グリッド上の点数より D+J が大きいです。")
    rng = random.Random(seed)
    grid = [(x, y) for x in range(1, num_rows_columns + 1) for y in range(1, num_rows_columns + 1)]
    rng.shuffle(grid)
    demand_points = grid[:D]
    candidate_sites = grid[D:D+J]
    return demand_points, candidate_sites

def compute_distances(demand_points: List[Point], candidate_sites: List[Point]) -> cp.ndarray:
    dp = cp.array(demand_points, dtype=float)  # (D,2)
    cs = cp.array(candidate_sites, dtype=float)  # (J,2)
    diff = dp[:, None, :] - cs[None, :, :]      # (D,J,2)
    return cp.sqrt((diff ** 2).sum(axis=2))     # (D,J)

def compute_wij_matrix(distances: cp.ndarray, alpha: float = 0.0, beta: float = 0.1) -> cp.ndarray:
    d = cp.asarray(distances, dtype=float)
    with cp.errstate(over='ignore'):
        w = cp.exp(-alpha * d) / (1.0 + beta * d)
    return w

def compute_Ui_from_set(wij: cp.ndarray, J_set: Iterable[int]) -> cp.ndarray:
    J_list = list(J_set)
    if len(J_list) == 0:
        return cp.zeros(wij.shape[0])
    return wij[:, J_list].sum(axis=1)  # D 次元

# ====== Stackelberg評価：w を直接使用（基礎効用=0 とみなす） ======
def leader_objective_from_sets(w: cp.ndarray, h: cp.ndarray, x_set: Set[int], y_set: Set[int]) -> float:
    """
    A_i = sum_{j in x} w_ij, B_i = sum_{j in y} w_ij
    L^+(x,y) = sum_i h_i * A_i / (A_i + B_i)
    """
    A = compute_Ui_from_set(w, x_set)
    B = compute_Ui_from_set(w, y_set)
    denom = A + B
    share_L = cp.where(denom > 0, A / denom, 0.0)
    return float((h * share_L).sum())

def competitor_objective_from_sets(w: cp.ndarray, h: cp.ndarray, x_set: Set[int], y_set: Set[int]) -> float:
    A = compute_Ui_from_set(w, x_set)
    B = compute_Ui_from_set(w, y_set)
    denom = A + B
    share_F = cp.where(denom > 0, B / denom, 0.0)
    return float((h * share_F).sum())

# ====== フォロワー最善応答 ======
def best_response_follower_exact(w: cp.ndarray, h: cp.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    best_y, best_val = set(), -inf
    available = [j for j in range(nJ) if j not in x_set]
    # 0..r すべて試す（小規模専用）
    for k in range(min(r, len(available)) + 1):
        for combo in combinations(available, k):
            y_set = set(combo)
            val = competitor_objective_from_sets(w, h, x_set, y_set)
            if val > best_val:
                best_val, best_y = val, y_set
    return best_y, best_val

def best_response_follower_greedy(w: cp.ndarray, h: cp.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    """
    大規模用：貪欲に r 回だけ周辺利得が最大のサイトを追加
    """
    y_set: Set[int] = set()
    candidates = [j for j in range(nJ) if j not in x_set]
    # 事前計算（現在のA,Bと分母を持っておく）
    A = compute_Ui_from_set(w, x_set)
    B = cp.zeros_like(A)

    for _ in range(min(r, len(candidates))):
        best_j, best_gain = None, -inf
        denom = A + B
        base_shareF = cp.where(denom > 0, B / denom, 0.0)
        base_val = float((h * base_shareF).sum())
        # 候補ごとの周辺利得
        for j in candidates:
            if j in y_set:
                continue
            Bj = B + w[:, j]
            denom_j = A + Bj
            shareF_j = cp.where(denom_j > 0, Bj / denom_j, 0.0)
            val_j = float((h * shareF_j).sum())
            gain = val_j - base_val
            if gain > best_gain:
                best_gain, best_j = gain, j
        if best_j is None:
            break
        y_set.add(best_j)
        B = B + w[:, best_j]
        candidates.remove(best_j)

    final_val = competitor_objective_from_sets(w, h, x_set, y_set)
    return y_set, final_val

"""
def best_response_of_competitor(w: cp.ndarray, h: cp.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    # サイズで厳密/貪欲を出し分け
    if nJ <= 25:
        return best_response_follower_exact(w, h, x_set, nJ, r)
    return best_response_follower_greedy(w, h, x_set, nJ, r)
"""
def best_response_of_competitor(w: cp.ndarray, h: cp.ndarray, x_set: Set[int], nJ: int, r: int) -> Tuple[Set[int], float]:
    # サイズで厳密/貪欲を出し分け
    #if nJ <= 25:
    return best_response_follower_exact(w, h, x_set, nJ, r)
    #return best_response_follower_greedy(w, h, x_set, nJ, r)

# ====== リーダー側の探索 ======
def solve_leader(w: cp.ndarray, h: cp.ndarray, nJ: int, p: int, r: int):
    """
    小規模：厳密全列挙
    大規模：前進貪欲（1 施設ずつ追加し、その都度フォロワー最善応答を反映）
    """
    
    
    # if nJ <= 25:
    best_val, best_x, best_y = -inf, None, None
    for combo in combinations(range(nJ), p):
        x_set = set(combo)
        y_set, _ = best_response_of_competitor(w, h, x_set, nJ, r)
        val = leader_objective_from_sets(w, h, x_set, y_set)
        if val > best_val:
            best_val, best_x, best_y = val, x_set, y_set
    return best_x, best_y, best_val

    """
    # 大規模：貪欲
    x_set: Set[int] = set()
    remaining = set(range(nJ))
    for _ in range(p):
        best_j, best_val = None, -inf
        for j in list(remaining):
            trial_x = set(x_set)
            trial_x.add(j)
            y_set, _ = best_response_of_competitor(w, h, trial_x, nJ, r)
            val = leader_objective_from_sets(w, h, trial_x, y_set)
            if val > best_val:
                best_val, best_j = val, j
        if best_j is None:
            break
        x_set.add(best_j)
        remaining.remove(best_j)
    # 最終 y と値
    y_set, _ = best_response_of_competitor(w, h, x_set, nJ, r)
    final_val = leader_objective_from_sets(w, h, x_set, y_set)
    return x_set, y_set, final_val
    """


In [6]:
instances = [
    {"I": 20,  "J": 20,  "p": 2, "r": 2},
    {"I": 20,  "J": 20,  "p": 3, "r": 2},
    {"I": 20,  "J": 20,  "p": 2, "r": 3},

    {"I": 40,  "J": 40,  "p": 2, "r": 2},
    {"I": 40,  "J": 40,  "p": 3, "r": 2},
    {"I": 40,  "J": 40,  "p": 2, "r": 3},

    {"I": 60,  "J": 60,  "p": 2, "r": 2},
    {"I": 60,  "J": 60,  "p": 3, "r": 2},
    {"I": 60,  "J": 60,  "p": 2, "r": 3},

    {"I": 80,  "J": 80,  "p": 2, "r": 2},
    {"I": 80,  "J": 80,  "p": 3, "r": 2},
    {"I": 80,  "J": 80,  "p": 2, "r": 3},

    {"I": 100, "J": 100, "p": 2, "r": 2},
    {"I": 100, "J": 100, "p": 3, "r": 2},
    {"I": 100, "J": 100, "p": 2, "r": 3},
]


In [7]:
n_runs = 1
num_rows_columns = 50   # グリッドの大きさ
alpha = 0
beta = 0.1


In [8]:
def make_seed(inst_id: int, run: int, base: int = 20250909) -> int:
    """インスタンスIDと試行番号に依存した再現可能シード"""
    return int(np.random.SeedSequence([base, inst_id, run]).generate_state(1)[0])

all_results = []

for inst_id, params in enumerate(instances, start=1):
    for run in range(n_runs):
        # ---- インスタンス生成 ----
        seed = make_seed(inst_id, run)
        demand_points, candidate_sites = generate_instance(
            num_rows_columns=num_rows_columns,
            D=params["I"], J=params["J"], seed=seed
        )

        # ---- 計算 ----
        distances = compute_distances(demand_points, candidate_sites)
        w = compute_wij_matrix(distances, alpha, beta)

        h_i = cp.full(params["I"], 1.0 / params["I"])
        x_star, y_star, L_val = solve_leader(w, h_i, nJ=params["J"], p=params["p"], r=params["r"])
        comp_val = competitor_objective_from_sets(w, h_i, x_star, y_star)

        row = {
            "instance_id": inst_id,
            "run": run,
            "I": params["I"], "J": params["J"], "p": params["p"], "r": params["r"],
            "leader_val": L_val,
            "comp_val": comp_val,
        }
        all_results.append(row)
        print(f"  試行 {run+1}/{n_runs} 完了: obj={L_val}")

# --- DataFrame 化 ---
df = pd.DataFrame(all_results)

# --- 平均・標準偏差を計算 ---
summary = (
    df.groupby(["instance_id", "D", "J", "p", "r"])
      .agg(
          leader_mean=("leader_val", "mean"),
          leader_std=("leader_val", "std"),
          comp_mean=("comp_val", "mean"),
          comp_std=("comp_val", "std"),
          n_runs=("run", "count"),
      )
      .reset_index()
)

# 保存
df.to_csv("results_all.csv", index=False, encoding="utf-8")
summary.to_csv("results_summary.csv", index=False, encoding="utf-8")

print("保存完了: results_all.csv (全試行), results_summary.csv (平均・標準偏差)")

RuntimeError: CuPy failed to load nvrtc64_120_0.dll: FileNotFoundError: Could not find module 'nvrtc64_120_0.dll' (or one of its dependencies). Try using the full path with constructor syntax.